In [137]:
import numpy as np
import pandas as pd
import random

train_pri_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")
train_df = train_pri_df.drop('Survived', axis=1)

titanic_all = pd.concat([train_pri_df, test_df])
print(train_pri_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcl

In [138]:
train_pri_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_pri_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.369942
2,"(32.252, 48.168]",0.404255
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


In [139]:
train_pri_df['AgeBand'] = pd.cut(train_df['Age'], 6)
train_pri_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 13.683]",0.591549
1,"(13.683, 26.947]",0.354839
2,"(26.947, 40.21]",0.428571
3,"(40.21, 53.473]",0.390000
4,"(53.473, 66.737]",0.348837
5,"(66.737, 80.0]",0.142857


In [158]:
age_group = train_pri_df.groupby(['SibSp', 'Parch', 'Pclass'])
age_mid = age_group.Age.median()
age_std = age_group.Age.std()

print(age_mid)

def guess_age(line):
  agm = age_mid.loc[(line['SibSp'], line['Parch'], line['Pclass'])]
  ags = age_std.loc[(line['SibSp'], line['Parch'], line['Pclass'])]
  if agm == 0.0 or ags==0.0:
    print(line['PassengerId'], line['SibSp'], line['Parch'], line['Pclass'])
  return random.uniform(agm - ags, agm + ags)

SibSp  Parch  Pclass
0      0      1         35.00
              2         30.00
              3         21.00
       1      1         37.00
              2         26.50
              3         18.00
       2      1         22.00
              2         13.00
              3          9.50
       3      3         24.00
       4      3         29.00
       5      3         40.00
1      0      1         35.00
              2         29.50
              3         19.50
       1      1         46.00
              2         23.50
              3         18.00
       2      1         14.00
              2         24.50
              3          0.50
       3      2         54.00
              3         32.00
       4      1         64.00
              3         42.50
       5      3         39.00
       6      3         43.00
2      0      1         50.00
              2         24.00
              3         21.00
       1      2         13.50
              3          0.75
       2      1    

In [157]:
train_pri_df.fillna(value={'Age':0}, inplace=True)

for idx, row in train_pri_df.iterrows():
  if row['Age'] == 0.0:
    row['Age'] = guess_age(row)

train_pri_df.info()

160 8 2 3
177 3 1 3
181 8 2 3
202 8 2 3
230 3 1 3
325 8 2 3
410 3 1 3
486 3 1 3
793 8 2 3
847 8 2 3
864 8 2 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    object  
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    object  
 12  AgeBand      714 non-null    category
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 84.8+ KB


In [155]:
mask = train_pri_df['PassengerId'].isin([160, 177, 181, 202, 230, 325, 410, 486, 793, 847, 864])

print(train_pri_df[mask])

print(train_pri_df.head())
print(train_pri_df.tail())

     PassengerId  Survived  Pclass                               Name     Sex  \
159          160         0       3         Sage, Master. Thomas Henry    male   
176          177         0       3      Lefebre, Master. Henry Forbes    male   
180          181         0       3       Sage, Miss. Constance Gladys  female   
201          202         0       3                Sage, Mr. Frederick    male   
229          230         0       3            Lefebre, Miss. Mathilde  female   
324          325         0       3           Sage, Mr. George John Jr    male   
409          410         0       3                 Lefebre, Miss. Ida  female   
485          486         0       3             Lefebre, Miss. Jeannie  female   
792          793         0       3            Sage, Miss. Stella Anna  female   
846          847         0       3           Sage, Mr. Douglas Bullen    male   
863          864         0       3  Sage, Miss. Dorothy Edith "Dolly"  female   

     Age  SibSp  Parch    T

In [86]:
def get_age_class(age):
  if age <= 14:
    ret_agg = 0
  elif 14 < age <= 27:
    ret_agg = 1
  elif 27 < age <= 40:
    ret_agg = 2
  elif 40 < age <= 53:
    ret_agg = 3
  elif 53 < age <= 67:
    ret_agg = 4
  else:
    ret_agg = 5
  return ret_agg


In [87]:
train_pri_df['Age'] = train_pri_df.apply(lambda x:get_age_class(x['Age']), axis=1)
print(train_pri_df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex  Age  SibSp  \
0                            Braund, Mr. Owen Harris    male    1      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female    2      1   
2                             Heikkinen, Miss. Laina  female    1      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female    2      1   
4                           Allen, Mr. William Henry    male    2      0   

   Parch            Ticket     Fare Cabin Embarked           AgeBand  \
0      0         A/5 21171   7.2500   NaN        S  (13.683, 26.947]   
1      0          PC 17599  71.2833   C85        C   (26.947, 40.21]   
2      0  STON/O2. 3101282   7.9250   NaN        S  (13.683, 26.947]   
3      0            113803  53.1000  C

In [89]:
train_pri_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    object  
 5   Age          891 non-null    int64   
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    object  
 12  AgeBand      714 non-null    category
 13  FareBand     891 non-null    category
dtypes: category(2), float64(1), int64(6), object(5)
memory usage: 85.9+ KB


In [72]:
train_pri_df['FareBand'] = pd.qcut(train_pri_df['Fare'], 4)
train_pri_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [73]:
train_pri_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    object  
 5   Age          714 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    object  
 12  AgeBand      714 non-null    category
 13  FareBand     891 non-null    category
dtypes: category(2), float64(2), int64(5), object(5)
memory usage: 85.9+ KB


In [99]:
df_pivot = pd.pivot_table(train_pri_df,
                    index=['Pclass', 'Sex'],
                    columns='AgeBand',
                    values=['Survived'],
                    aggfunc=['mean'])
df_pivot

mean                                                   \
                    Survived                                                    
AgeBand       (0.34, 13.683] (13.683, 26.947] (26.947, 40.21] (40.21, 53.473]   
Pclass Sex                                                                      
1      female           0.00         0.964286        1.000000        0.941176   
       male             1.00         0.416667        0.500000        0.392857   
2      female           1.00         0.904762        0.935484        0.900000   
       male             1.00         0.035714        0.073171        0.083333   
3      female           0.50         0.522727        0.440000        0.000000   
       male             0.36         0.113043        0.170732        0.080000   

                                               
                                               
AgeBand       (53.473, 66.737] (66.737, 80.0]  
Pclass Sex                                     
1      female         1.000000            NaN  
       male           0.111111           0.25  
2      female         0.666667            NaN  
       male           0.125000           0.00  
3      female         1.000000            NaN  
       male           0.000000           0.00

In [101]:
age_group = train_pri_df.groupby(['AgeBand', 'Pclass'])
print(age_group.PassengerId.count())

AgeBand           Pclass
(0.34, 13.683]    1           4
                  2          18
                  3          49
(13.683, 26.947]  1          40
                  2          49
                  3         159
(26.947, 40.21]   1          66
                  2          72
                  3         107
(40.21, 53.473]   1          45
                  2          22
                  3          33
(53.473, 66.737]  1          27
                  2          11
                  3           5
(66.737, 80.0]    1           4
                  2           1
                  3           2
Name: PassengerId, dtype: int64


In [53]:
age_group = train_pri_df.groupby(['SibSb', 'Parch'])
print(age_group.Age.sum())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [54]:
print(titanic_all.loc[:, ['Pclass', 'Cabin']])
titanic_all.drop(['Cabin', 'Ticket'], axis=1, inplace=True)
titanic_all.info()

     Pclass Cabin
0         3   NaN
1         1   C85
2         3   NaN
3         1  C123
4         3   NaN
..      ...   ...
413       3   NaN
414       1  C105
415       3   NaN
416       3   NaN
417       3   NaN

[1309 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   int64  
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Fare         1308 non-null   float64
 9   Embarked     1307 non-null   object 
dtypes: float64(3), int64(5), object(2)
memory usage: 112.5+ KB


In [14]:
titanic_all = pd.concat([train_df, test_df])
print(titanic_all.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB
None


In [15]:
print(titanic_all.head())
print(titanic_all.tail())

   PassengerId  Pclass                                               Name  \
0            1       3                            Braund, Mr. Owen Harris   
1            2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3       3                             Heikkinen, Miss. Laina   
3            4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4            5       3                           Allen, Mr. William Henry   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S  
1  female  38.0      1      0          PC 17599  71.2833   C85        C  
2  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S  
3  female  35.0      1      0            113803  53.1000  C123        S  
4    male  35.0      0      0            373450   8.0500   NaN        S  
     PassengerId  Pclass                          Name     Sex   Age  SibSp  \
413         13